In [1]:
# INVESTIGATING LATENT FUNCTONAL NETWORKS IN THE BRAIN DURING EMOTION PERCEPTION

# Project Completed as a part of NeuroMatch Academy 2022 - Computational Neuroscience course


# Pod Name: Cachupa
#  Members: 
# - Rohit Misra 
# - Iosif Lytras
# - Iraj Ahangari 
# - Gürkan Sinan Yaşar
# - Dong Chenjie 

#  Teaching Assistants: 
# - Bindu Kumari
# - Tanya Upadhyay

#  Mentors:
# - Maria Eckstein
# - Margarita Zachariou




**SET-UP**

In [2]:
from distutils import filelist
import numpy as np
import nibabel
from nilearn import datasets
# from nilearn.maskers import NiftiMapsMasker
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, group_sparse_covariance
import scipy.io
from nilearn import plotting
import matplotlib.pyplot as plt
import os
import networkx as nx
from networkx.algorithms import community
import pandas as pd
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import community as community_louvain
from sklearn.decomposition import PCA

**FUNCTION DEFINITONS**

Functions to return file path given subject and run number

In [3]:
# function to return file destination for given subject number and run  number
def fmri_filename(subject_number, run_number):
    filename_prefix = "/kaggle/input/fmri-dataset-for-emotion-recognition/"
    filename_suffix = "Sub-0" + str(subject_number) + "/wrsub-0" + str(subject_number) + "_task-emotionalfaces_run-" + str(run_number) + "_bold.nii"
    return filename_prefix + filename_suffix
    
def get_events_filename(run_number):
    filename = "/kaggle/input/events/events/task-emotionalfaces_run-" + str(run_number) + "_events.tsv"
    return filename

Function to read fMRI file and generate parcellated time series using given atlas

In [4]:
def getParcellations(fmri_filename, atlas):
    # Loading atlas image stored in 'maps'
    atlas_filename = atlas.maps #atlas['maps']
    # Loading atlas data stored in 'labels'
    labels = atlas.labels #atlas['labels']
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                            memory='nilearn_cache', verbose=0)
    # masker.fit(data)
    time_series = masker.fit_transform(fmri_filename)
    return [time_series,labels]

Functions to segment the ROI time series based on stimulus and generate connectivity matrices for each segment

In [5]:
def get_correlation_matrices(subject_number, run_number, atlas):
    bold_filename = fmri_filename(subject_number, run_number)
    events_filename = get_events_filename(run_number)
    [ROI_time_series, ROI_labels] = getParcellations(bold_filename, atlas)
    blocks_time_series, stimulus_labels = segment_time_series(ROI_time_series, events_filename)
    
    correlation_matrices = np.ndarray((blocks_time_series[0].shape[1], blocks_time_series[0].shape[1], len(stimulus_labels)))
    correlation_measure = ConnectivityMeasure(kind='correlation')
    for i, time_series in enumerate(blocks_time_series):
        block_corr_matrix = correlation_measure.fit_transform([time_series])[0]
        correlation_matrices[:,:,i] = block_corr_matrix
    return [correlation_matrices, stimulus_labels]
    
def segment_time_series(time_series, events_filename):
    blocks_time_series = []
    blocks_labels = []

    events_df = pd.read_csv(events_filename,sep="\t")
    events_list = events_df.iloc[:,2].tolist()
    stim_labels_list = ['happy', 'sad', 'angry', 'neutral']
    for stimulus in stim_labels_list:
        onset_instances = [i for i,label in enumerate(events_list) if label == stimulus]
        for onset_index in onset_instances:
            start = events_df.iloc[onset_index,0]//2
            stop = start + (events_df.iloc[onset_index,1]//2) - 1
            blocks_time_series.append(time_series[start:stop,:])
            blocks_labels.append(stimulus)
    return [blocks_time_series, blocks_labels]

Functions to Vectorize the connectivity matrices, and convert vectors to connectivity matrices

In [6]:
def corr_matrices_to_vectors(correlation_matrices):
    n = correlation_matrices.shape[0]
    p = int(n * (n - 1) / 2)
    vectors = np.ndarray((correlation_matrices.shape[2],p))
    start = 0
    for iter in range(n):
        row = correlation_matrices[iter, iter+1:, :].T
        end = start + row.shape[1]
        vectors[:,start:end] = row
        start = end
    return vectors


def vector_to_matrix(vector, N):
    matrix = np.zeros((N,N))
    start = 0
    for iter in range(N-1):
        end = start + N - 1 - iter
        snippet = vector[start:end]
        matrix[iter, iter+1:] = snippet
        matrix[iter+1:, iter] = snippet.T
        start = end
    return matrix


Functions to cluster data using k-means algorithm for different k values and compare them based on silhouette score

In [7]:
# data is a matrix of size "number of blocks x (n^2 - n)/2"

def k_means_score(data, k):
    km = KMeans(n_clusters=k, random_state=37)
    km.fit(data)
    pred = km.predict(data)
    return [pred, silhouette_score(data, pred)]

def compare_k_means(data, k_min, k_max):
    scores = []
    k_values = np.arange(k_min, k_max+1)
    for k in range(k_min, k_max + 1):
        _, score = k_means_score(data, k)
        scores.append(score)
    return [k_values, scores]

def optimal_k_means(data, k):
    km = KMeans(n_clusters=k, random_state=37)
    km.fit(data)
    pred = km.predict(data)
    centroids = km.cluster_centers_
    return [pred, centroids]



Functions to perform graph theoretic analysis of matrices

In [ ]:
def corr_to_adj(M):
    threshold = 0.3
    M[np.abs(M) < threshold] = 0
    return M

def graph_analysis(A):
    G = nx.from_numpy_array(A)
    # print(G)
    pos = nx.shell_layout(G)
    # nx.draw(G, pos = pos)
    # plt.show()
    global_efficiency = nx.global_efficiency(G)
    partitions = community_louvain.best_partition(G)
    # print(partitions)
    modularity = community_louvain.modularity(partitions, G)
    return [global_efficiency, modularity]



Functions to plot the connectivity matrix and the connectoome given a matrix

In [ ]:
def get_HO_coords(atlas):
    atlas_img = atlas['maps']
    # all ROIs except the background
    values = np.unique(atlas_img.get_data())[1:] 
    # iterate over Harvard-Oxford ROIs
    coords = []
    for v in values:
        data = np.zeros_like(atlas_img.get_data())
        data[atlas_img.get_data() == v] = 1
        xyz = plotting.find_xyz_cut_coords(nibabel.Nifti1Image(data, atlas_img.affine))
        coords.append(xyz)
    return coords

def plotCorrelationMatrix(correlation_matrix,atlas):
    labels = atlas.labels[1:]
    np.fill_diagonal(correlation_matrix, 0)
    plotting.plot_matrix(correlation_matrix,  labels = labels, colorbar=True,
                     vmax=1, vmin=0.3)
#     coords = atlas.region_coords
    
    # only for harvard oxford atlas 
    coords = get_HO_coords(atlas)
    plotting.plot_connectome(correlation_matrix, coords,
                         edge_threshold="95%", colorbar=True)

    plotting.show()
    view = plotting.view_connectome(correlation_matrix, coords, edge_threshold='90%')
#     view.open_in_browser()

Function to perform graph analysis on given matrix

In [ ]:
def rep_network_analysis(matrix):
#     plotCorrelationMatrix(matrix, atlas)
    adj_matrix = corr_to_adj(matrix)
    global_efficiency, modularity = graph_analysis(adj_matrix)
    print("Graph Global Efficieny = " + str(global_efficiency))
    print("Graph Modularity= " + str(modularity))
    return [global_efficiency, modularity]
    

**MAIN**

1. Iterate over all fMRI files and generate correlation matrices for all blocks in each run. 

In [ ]:
# total_subjects = 5
# total_runs_per_subject = 5

# # atlas = datasets.fetch_atlas_aal(version='SPM12', data_dir=None, url=None, resume=True, verbose=1)
# atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
# n = len(atlas.labels)
# all_correlation_matrices = np.ndarray((n-1,n-1))
# all_stimulus_labels = []
# for subject_num in range(1,total_subjects + 1):
#     for run_num in range(1,total_runs_per_subject + 1):
#         [runwise_corr_matrices, stimulus_labels] = get_correlation_matrices(subject_num, run_num, atlas)
#         all_correlation_matrices = np.dstack((all_correlation_matrices, runwise_corr_matrices))
#         all_stimulus_labels.extend(stimulus_labels)
        
# all_correlation_matrices = all_correlation_matrices[:,:,1:]
# print(all_correlation_matrices.shape)


2. Vectorize the connectivity matrices

In [ ]:
# vectorized_data = corr_matrices_to_vectors(all_correlation_matrices)
# print(vectorized_data.shape)

3. Perform PCA to reduce dimensionality of the vectors

In [ ]:
# pca =  PCA(n_components= 100)
# pca.fit(vectorized_data)
# data = pca.transform(vectorized_data)

4. Cluster the data using k-means clustering for a range of k-values

In [ ]:
# k_values, scores = compare_k_means(data, 2, 15)

# plt.plot(k_values, scores)
# plt.xlabel('k value')
# plt.ylabel('Silhouette score')
# plt.show()

# # pick max

5. Find the k that maximises silhouette score

In [ ]:
# k_opt = k_values[np.argmax(scores)]
# print(k_opt)

6. Convert cluster centroids to connectivity matrices to get representative networks 

In [ ]:
# cluster_labels, centroids_pca = optimal_k_means(data, k_opt)
# centroids = pca.inverse_transform(centroids_pca)
# N = len(atlas.labels[1:])
# rep_networks = np.ndarray((N,N,centroids.shape[0]))
# for index in range(centroids.shape[0]):
#     rep_networks[:,:,index] = vector_to_matrix(centroids[index,:], N)
    
# print(rep_networks.shape)

7. Generate graph theoretic metrics for all represenatative networks

In [9]:
# global_efficiency_list = []
# modularity_list = []
# for index in range(rep_networks.shape[2]):
#     eff, mod = rep_network_analysis(rep_networks[:,:,index])
#     global_efficiency_list.append(eff)
#     modularity_list.append(mod)
    


8. (A) Compare graph metrics using bar graphs

In [10]:
# network_index = np.arange(k_opt) + 1

# plt.bar(network_index, global_efficiency_list) 
# plt.xlabel("Representative Networks")
# plt.ylabel("Global Efficiency")
# plt.title("Global Efficiency of Representative Networks")
# plt.savefig("eff_bar")
# plt.show()

8. (B) Compare graph metrics using bar graphs

In [11]:
# plt.bar(network_index, modularity_list) 
# plt.xlabel("Representative Networks")
# plt.ylabel("Modularity")
# plt.title("Modularity of Representative Networks")
# plt.savefig("modularity_bar")
# plt.show()

9. (A) Display Representative networks

In [12]:
# plotCorrelationMatrix(rep_networks[:,:,0], atlas)

9. (B) Display Representative networks

In [13]:
# plotCorrelationMatrix(rep_networks[:,:,1], atlas)


10. Visualise unclustered data using t-SNE

In [14]:
# # visualization with tSNE in two dinemsions
# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, verbose=1, random_state=123)
# embed_2D = tsne.fit_transform(vectorized_data)

# component_1 = embed_2D[:,0]
# component_2 = embed_2D[:,1]


# plt.scatter(component_1, component_2)
# plt.xlabel('component 1')
# plt.ylabel('component 2')


11. Get composition of clusters based on experimenter labels

In [15]:
# stim_labels_list = ['happy', 'sad', 'angry', 'neutral']
# composition = np.zeros((len(np.unique(cluster_labels)), len(stim_labels_list)))
# for i in range(len(cluster_labels)):
#     composition[cluster_labels[i], stim_labels_list.index(all_stimulus_labels[i])] += 1
    
# print(composition)